In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"belalsaeid","key":"cb5572dba32108d95580109d6e1b200c"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset
from zipfile import ZipFile

# Unzip the downloaded dataset
with ZipFile("plantvillage-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall()

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
 99% 2.02G/2.04G [00:14<00:00, 43.3MB/s]
100% 2.04G/2.04G [00:14<00:00, 146MB/s] 


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio tensorflow pillow numpy python-multipart

from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Define constants
img_size = 256
batch_size = 32
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/model_epoch_20.h5"

# Create and compile the model
def create_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=[img_size, img_size, 3]),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),  # New dropout layer added to reduce overfitting
        layers.Dense(num_classes, activation='softmax')
    ])
    from tensorflow.keras.optimizers import Adam
    optimizer = Adam(learning_rate=0.0001)  # Lower learning rate for more stable training
    model.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')
    return model


# Image preprocessing function
def load_and_preprocess(image_path):
    img = Image.open(image_path).convert('RGB').resize([img_size, img_size])
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255
    return img_array

def predict_image_class(model, image_path, class_indices):
    preprocess_img = load_and_preprocess(image_path)
    predictions = model.predict(preprocess_img)[0]
    predicted_class_index = np.argmax(predictions)
    return class_indices[predicted_class_index]


# Setup data generators
def setup_data_generators():
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=20,          # Randomly rotate images by up to 20 degrees
        width_shift_range=0.1,        # Randomly shift images horizontally by 10%
        height_shift_range=0.1,       # Randomly shift images vertically by 10%
        shear_range=0.1,              # Apply shearing transformations
        zoom_range=0.1,               # Randomly zoom in on images
        horizontal_flip=True          # Randomly flip images horizontally
    )
    dataset_path = "/content/plantvillage dataset/color"
    train_gen = datagen.flow_from_directory(
        dataset_path, target_size=(img_size, img_size), batch_size=batch_size,
        class_mode='categorical', subset='training'
    )
    validation_gen = datagen.flow_from_directory(
        dataset_path, target_size=(img_size, img_size), batch_size=batch_size,
        class_mode='categorical', subset='validation'
    )
    return train_gen, validation_gen


# Create FastAPI app with CORS enabled
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins (replace "*" with your web app's domain for production)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Set this flag to True if you want to train the model for additional epochs in this session.
# Set it to False if you only want to load the saved model for the API.
should_train = False  # Change to True to train for additional epochs

if should_train:
# Setup data generators
    train_gen, validation_gen = setup_data_generators()

    # If a saved model exists, load it. Otherwise, create a new one.
    if os.path.exists(MODEL_PATH):
        print("Loading saved model for training...")
        model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("Training model from scratch...")
        model = create_model(train_gen.num_classes)

    # Define number of epochs
    epochs = 100

    # Setup callbacks
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/Colab Notebooks/model_epoch_{epoch:02d}.h5",
    save_best_only=False,
    save_weights_only=False
)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    try:
        # Train the model
        model.fit(
            train_gen,
            steps_per_epoch=train_gen.samples // batch_size,
            epochs=epochs,
            validation_data=validation_gen,
            validation_steps=validation_gen.samples // batch_size,
            callbacks=[checkpoint_cb, early_stop]
        )
    except KeyboardInterrupt:
        print("Training interrupted manually. Saving model...")
        model.save(MODEL_PATH)
        print("Model saved.")
else:
    # Setup data generators for inference
    train_gen, _ = setup_data_generators()

    # If not training, just load the model for API usage.
    if os.path.exists(MODEL_PATH):
        print("Loading saved model for API usage...")
        model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("Model not found. Training model from scratch (only for API usage)...")
        model = create_model(train_gen.num_classes)


# Create class indices for mapping predictions
class_indices = {v: k for k, v in train_gen.class_indices.items()}

# Define the prediction endpoint
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        temp_dir = "temp_uploads"
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, file.filename)
        with open(temp_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
        predict_name = predict_image_class(model, temp_path, class_indices)
        data = predict_name.split("___")
        info = {
            "name": " ".join(data[0].split("_")),
            "condition": " ".join(data[1].split("_"))
        }
        return info
    except Exception as e:
        return JSONResponse(status_code=500, content={"message": f"Error processing image: {str(e)}"})


# Start the server with ngrok
nest_asyncio.apply()
ngrok.set_auth_token("2yMMFAjAP2weUpE1Xj4cg5WHaBj_2SqYRUmg5rW1vYmWt1CL3")
public_url = ngrok.connect(8000)
print(f"API is running at: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-23' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    s

Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.
Training model from scratch...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
 460/1358 ━━━━━━━━━━━━━━━━━━━━ 1:14 83ms/step - accuracy: 0.2659 - loss: 2.8973

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 580, in run
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    se

...

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 129s 95ms/step - accuracy: 0.9439 - loss: 0.1683 - val_accuracy: 0.9113 - val_loss: 0.2883
Epoch 19/100
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9466 - loss: 0.1586

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 135s 90ms/step - accuracy: 0.9466 - loss: 0.1586 - val_accuracy: 0.9192 - val_loss: 0.2629
Epoch 20/100
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.9518 - loss: 0.1486

1358/1358 ━━━━━━━━━━━━━━━━━━━━ 132s 97ms/step - accuracy: 0.9518 - loss: 0.1486 - val_accuracy: 0.9213 - val_loss: 0.2591
API is running at: NgrokTunnel: "https://8964-34-125-217-213.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [550]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
INFO:     197.51.155.117:0 - "POST /predict/ HTTP/1.1" 200 OK


In [6]:
!pip install fastapi uvicorn pyngrok nest_asyncio tensorflow pillow numpy python-multipart

from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn
from pyngrok import ngrok
import os
import random
import shutil
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Define constants
img_size = 256
batch_size = 32
MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/model_epoch_20.h5"

# Create and compile the model
def create_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=[img_size, img_size, 3]),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),  # New dropout layer added to reduce overfitting
        layers.Dense(num_classes, activation='softmax')
    ])
    from tensorflow.keras.optimizers import Adam
    optimizer = Adam(learning_rate=0.0001)  # Lower learning rate for more stable training
    model.compile(optimizer=optimizer, metrics=['accuracy'], loss='categorical_crossentropy')
    return model


# Image preprocessing function
def load_and_preprocess(image_path):
    img = Image.open(image_path).convert('RGB').resize([img_size, img_size])
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255
    return img_array

def predict_image_class(model, image_path, class_indices):
    preprocess_img = load_and_preprocess(image_path)
    predictions = model.predict(preprocess_img)[0]
    predicted_class_index = np.argmax(predictions)
    return class_indices[predicted_class_index]


# Setup data generators
def setup_data_generators():
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        rotation_range=20,          # Randomly rotate images by up to 20 degrees
        width_shift_range=0.1,        # Randomly shift images horizontally by 10%
        height_shift_range=0.1,       # Randomly shift images vertically by 10%
        shear_range=0.1,              # Apply shearing transformations
        zoom_range=0.1,               # Randomly zoom in on images
        horizontal_flip=True          # Randomly flip images horizontally
    )
    dataset_path = "/content/plantvillage dataset/color"
    train_gen = datagen.flow_from_directory(
        dataset_path, target_size=(img_size, img_size), batch_size=batch_size,
        class_mode='categorical', subset='training'
    )
    validation_gen = datagen.flow_from_directory(
        dataset_path, target_size=(img_size, img_size), batch_size=batch_size,
        class_mode='categorical', subset='validation'
    )
    return train_gen, validation_gen


# Create FastAPI app with CORS enabled
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins (replace "*" with your web app's domain for production)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Set this flag to True if you want to train the model for additional epochs in this session.
# Set it to False if you only want to load the saved model for the API.
should_train = False  # Change to True to train for additional epochs

if should_train:
# Setup data generators
    train_gen, validation_gen = setup_data_generators()

    # If a saved model exists, load it. Otherwise, create a new one.
    if os.path.exists(MODEL_PATH):
        print("Loading saved model for training...")
        model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("Training model from scratch...")
        model = create_model(train_gen.num_classes)

    # Define number of epochs
    epochs = 100

    # Setup callbacks
    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/MyDrive/Colab Notebooks/model_epoch_{epoch:02d}.h5",
    save_best_only=False,
    save_weights_only=False
)
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    try:
        # Train the model
        model.fit(
            train_gen,
            steps_per_epoch=train_gen.samples // batch_size,
            epochs=epochs,
            validation_data=validation_gen,
            validation_steps=validation_gen.samples // batch_size,
            callbacks=[checkpoint_cb, early_stop]
        )
    except KeyboardInterrupt:
        print("Training interrupted manually. Saving model...")
        model.save(MODEL_PATH)
        print("Model saved.")
else:
    # Setup data generators for inference
    train_gen, _ = setup_data_generators()

    # If not training, just load the model for API usage.
    if os.path.exists(MODEL_PATH):
        print("Loading saved model for API usage...")
        model = tf.keras.models.load_model(MODEL_PATH)
    else:
        print("Model not found. Training model from scratch (only for API usage)...")
        model = create_model(train_gen.num_classes)


# Create class indices for mapping predictions
class_indices = {v: k for k, v in train_gen.class_indices.items()}

# Define the prediction endpoint
@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    try:
        temp_dir = "temp_uploads"
        os.makedirs(temp_dir, exist_ok=True)
        temp_path = os.path.join(temp_dir, file.filename)
        with open(temp_path, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)
        predict_name = predict_image_class(model, temp_path, class_indices)
        data = predict_name.split("___")
        info = {
            "name": " ".join(data[0].split("_")),
            "condition": " ".join(data[1].split("_"))
        }
        return info
    except Exception as e:
        return JSONResponse(status_code=500, content={"message": f"Error processing image: {str(e)}"})


# Start the server with ngrok
nest_asyncio.apply()
ngrok.set_auth_token("2yMMFAjAP2weUpE1Xj4cg5WHaBj_2SqYRUmg5rW1vYmWt1CL3")
public_url = ngrok.connect(8000)
print(f"API is running at: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.
Loading saved model for API usage...


API is running at: NgrokTunnel: "https://8485-34-106-71-238.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [463]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
INFO:     197.51.16.212:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
INFO:     197.51.16.212:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
INFO:     197.51.16.212:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
INFO:     197.51.16.212:0 - "POST /predict/ HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
INFO:     197.51.16.212:0 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [463]
